# Segmentacja obiektów pierwszoplanowych
## Metody oparte o bufor próbek

In [1]:
import numpy as np
import cv2

### Metody pomocnicze

In [2]:
def I_gray(filename):
    I = cv2.imread(filename)
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    return I

In [3]:
def I_original(filename):
    I = cv2.imread(filename)
    return I

In [4]:
def I_diff_and_thresh(I_prev, I, thresh):
    I_abs_diff = cv2.absdiff(I_prev, I)
    (T, I_thresh) = cv2.threshold(I_abs_diff, thresh,255,cv2.THRESH_BINARY)
    return I_thresh

In [5]:
def I_indexation(I_input, I):
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(I)

    I_VIS = I_input # copy of the input image
    if (stats.shape[0] > 1): # are there any objects
        tab = stats[1:,4] # 4 columns without first element
        pi = np.argmax( tab )# finding the index of the largest item
        pi = pi + 1 # increment because we want the index in stats, not in tab
        # drawing a bbox
        cv2.rectangle(I_VIS,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],
        stats[pi,1]+stats[pi,3]),(255,0,0),2)
        cv2.putText(I_VIS,"%f" % stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))
        cv2.putText(I_VIS,"%d" %pi,(int(centroids[pi,0]),int(centroids[pi,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    return I_VIS, np.uint8(labels / retval * 255)

In [6]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 3)

    kernel = np.ones((3,3),np.uint8)
    I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_median,kernel)

    return I_result

### Implementacja

In [8]:
TP, TN, FP, FN = 0, 0, 0, 0

In [9]:
N = 60
I = I_gray('../common/pedestrian/input/in%06d.jpg' % 1)

YY, XX = I.shape
BUF = np.zeros((YY,XX,N),np.uint8)
iN = 0

### Średnia

In [10]:
def process_video(folder, start_frame, end_frame):
    iN = 0
    TP, TN, FP, FN = 0, 0, 0, 0

    I = I_gray(folder + '/input/in%06d.jpg' % 1)
    YY, XX = I.shape
    BUF = np.zeros((YY,XX,N),np.uint8)
    for i in range(start_frame, end_frame, 2):
        IG = I_gray(folder + '/input/in%06d.jpg' % i)
        BUF[:,:,iN] = IG
        iN = (iN + 1) % 60
        mean = (np.mean(BUF, axis=2)).astype(np.uint8)
        I_ref = I_gray(folder + '/groundtruth/gt%06d.png' % i)
        I_color = I_original(folder + '/input/in%06d.jpg' % i)
        I_result = I_diff_and_thresh(mean, IG, 20)
        cv2.imshow("I_result", I_result)

        I_result_gray = I__median_and_morfology(I_result)

        I_result, tmp = I_indexation(I_color, I_result_gray)
        cv2.imshow("I_result_gray", I_result_gray)
        cv2.waitKey(10)
        f = open(folder + '/temporalROI.txt','r') # open file
        line = f.readline() # read line
        roi_start, roi_end = line.split() # split line
        roi_start = int(roi_start) # conversion to int
        roi_end = int(roi_end) # conversion to int

        if (i >= roi_start and i <= roi_end):
            TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
            TP_S = np.sum(TP_M) # sum of the elements in the matrix
            TP = TP + TP_S # update of the global indicator

            TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
            TN_S = np.sum(TN_M) # sum of the elements in the matrix
            TN = TN + TN_S # update of the global indicator

            FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
            FP_S = np.sum(FP_M) # sum of the elements in the matrix
            FP = FP + FP_S # update of the global indicator

            FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
            FN_S = np.sum(FN_M) # sum of the elements in the matrix
            FN = FN + FN_S # update of the global indicator
    cv2.destroyAllWindows()

    P = TP / (TP + FP)
    R = TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print(F1)

Pedestrian

In [27]:
folder = '../common/pedestrian'
process_video(folder, 300, 1100)

0.12179949155069279


Highway

In [28]:
folder = '../common/highway'
process_video(folder, 470, 1701)

0.43589317338095923


In [29]:
folder = '../common/office'
process_video(folder, 570, 2050)

KeyboardInterrupt: 

### Mediana

In [30]:
I = I_gray('../common/pedestrian/input/in%06d.jpg' % 299)

YY, XX = I.shape
BUF = np.zeros((YY,XX,N),np.uint8)
iN = 0

for i in range(300,1100, 2):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)
    BUF[:,:,iN] = IG
    iN = (iN + 1) % 60
    median = (np.median(BUF, axis=2)).astype(np.uint8)
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/pedestrian/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(median, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.20054060005675836

In [31]:
I = I_gray('../common/highway/input/in%06d.jpg' % 469)

YY, XX = I.shape
BUF = np.zeros((YY,XX,N),np.uint8)
iN = 0

for i in range(470,1701, 2):
    IG = I_gray('../common/highway/input/in%06d.jpg' % i)
    BUF[:,:,iN] = IG
    iN = (iN + 1) % 60
    median = (np.median(BUF, axis=2)).astype(np.uint8)
    I_ref = I_gray('../common/highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/highway/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(median, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result_gray", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.5306818799097345

In [ ]:
I = I_gray('../common/office/input/in%06d.jpg' % 569)

YY, XX = I.shape
BUF = np.zeros((YY,XX,N),np.uint8)
iN = 0

for i in range(570,2050, 3):
    IG = I_gray('../common/office/input/in%06d.jpg' % i)
    BUF[:,:,iN] = IG
    iN = (iN + 1) % 60
    median = (np.median(BUF, axis=2)).astype(np.uint8)
    I_ref = I_gray('../common/office/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/office/input/in%06d.jpg' % i)
    I_result = I_diff_and_thresh(median, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result_gray", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.6592022450461996

## Aproksymacja sredniej i mediany (tzw. sigma-delta)
### Średnia
Pedestrian

In [8]:
alfa = 0.05

In [41]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 3)

    kernel = np.ones((3,3),np.uint8)
    I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_result,kernel)

    return I_result

In [42]:
BG = I_gray('../common/pedestrian/input/in%06d.jpg' % 299)

for i in range(300,1100):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/pedestrian/input/in%06d.jpg' % i)
    BG = alfa * IG + (1 - alfa) * BG
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result_gray", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.5406136887232686

Highway

In [43]:
BG = I_gray('../common/highway/input/in%06d.jpg' % 469)

for i in range(470,1701, 2):
    IG = I_gray('../common/highway/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/highway/input/in%06d.jpg' % i)
    BG = alfa * IG + (1 - alfa) * BG
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result_gray", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.5667646777184131

Office

In [ ]:
BG = I_gray('../common/office/input/in%06d.jpg' % 569)

for i in range(570, 2051):
    IG = I_gray('../common/office/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/office/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/office/input/in%06d.jpg' % i)
    BG = alfa * IG + (1 - alfa) * BG
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.7101643562433197

### Mediana
Pedestrian

In [13]:
def median(BG, I):
    if ((BG < I).sum() == 0):
        return BG + 1
    elif ((BG > I).sum() == 0):
        return BG - 1
    else:
        return BG

In [49]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 3)

    kernel = np.ones((3,3),np.uint8)
    # I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_median,kernel)

    return I_result

In [47]:
BG = I_gray('../common/pedestrian/input/in%06d.jpg' % 299)

for i in range(300,1100, 3):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/pedestrian/input/in%06d.jpg' % i)
    BG = median(BG, IG)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result_gray", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.5762357607833148

Highway

In [ ]:
BG = I_gray('../common/highway/input/in%06d.jpg' % 469)

for i in range(470,1701):
    IG = I_gray('../common/highway/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/highway/input/in%06d.jpg' % i)
    BG = median(BG, IG)
    I_result = I_diff_and_thresh(BG, IG, 35)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.7853635214080669

Office

In [ ]:
BG = I_gray('../common/office/input/in%06d.jpg' % 569)

for i in range(570,2051, 2):
    IG = I_gray('../common/office/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/office/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/office/input/in%06d.jpg' % i)
    BG = median(BG, IG)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_result", I_result)

    I_result_gray = I__median_and_morfology(I_result)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    cv2.waitKey(10)
    f = open('../common/office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.7693424302886936

## Podejście konserwatywne

### Średnia
Pedestrian

In [11]:
BG = I_gray('../common/pedestrian/input/in%06d.jpg' % 299)
TP, TN, FP, FN = 0, 0, 0, 0

for i in range(300,1100, 2):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/pedestrian/input/in%06d.jpg' % i)
    if (i == 300):
        BG = alfa * IG + (1 - alfa) * BG
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = alfa * IG[mask] + (1 - alfa) * BG[mask]
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_diff", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 600):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.6386226072597242

Highway

In [ ]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 5)

    kernel = np.ones((3,3),np.uint8)
    I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_result,kernel)

    return I_result

In [55]:
BG = I_gray('../common/highway/input/in%06d.jpg' % 469)

for i in range(470,1701, 2):
    IG = I_gray('../common/highway/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/highway/input/in%06d.jpg' % i)
    if (i == 470):
        BG = alfa * IG + (1 - alfa) * BG
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = alfa * IG[mask] + (1 - alfa) * BG[mask]
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 55)
    cv2.imshow("I_diff", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_morfology", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 1200):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.6255617179853603

Office

In [56]:
BG = I_gray('../common/office/input/in%06d.jpg' % 569)

for i in range(570,2051, 2):
    IG = I_gray('../common/office/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/office/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/office/input/in%06d.jpg' % i)
    cv2.imshow("I_result", BG)
    if (i == 570):
        BG = alfa * IG + (1 - alfa) * BG
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = alfa * IG[mask] + (1 - alfa) * BG[mask]
    BG = BG.astype(np.uint8)
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_gray", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 600):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

F1

0.5781212148312452

### Mediana
Pedestrian

In [14]:
BG = I_gray('../common/pedestrian/input/in%06d.jpg' % 299)

for i in range(300,1100, 3):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/pedestrian/input/in%06d.jpg' % i)
    if (i == 300):
        BG = median(BG, IG)
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = median(BG[mask], IG[mask])
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_gray", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 600):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

In [59]:
F1

0.5830511928964139

Highway

In [60]:
BG = I_gray('../common/highway/input/in%06d.jpg' % 469)

for i in range(470, 1701, 2):
    IG = I_gray('../common/highway/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/highway/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/highway/input/in%06d.jpg' % i)
    if (i == 470):
        BG = median(BG, IG)
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = median(BG[mask], IG[mask])
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_gray", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 1200):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/highway/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.6224841541608528

Office

In [61]:
BG = I_gray('../common/office/input/in%06d.jpg' % 569)

for i in range(570, 2051, 2):
    IG = I_gray('../common/office/input/in%06d.jpg' % i)
    I_ref = I_gray('../common/office/groundtruth/gt%06d.png' % i)
    I_color = I_original('../common/office/input/in%06d.jpg' % i)
    if (i == 570):
        BG = median(BG, IG)
    else:
        mask = np.logical_and(I_result_gray, IG)
        BG[mask] = median(BG[mask], IG[mask])
    I_result = I_diff_and_thresh(BG, IG, 35)
    cv2.imshow("I_gray", I_result)

    I_result_gray = I__median_and_morfology(I_result)
    cv2.imshow("I_result", I_result_gray)

    I_result, tmp = I_indexation(I_color, I_result_gray)
    if (i == 600):
        cv2.waitKey(0)
    cv2.waitKey(10)
    f = open('../common/office/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((I_result_gray == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((I_result_gray == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((I_result_gray == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((I_result_gray == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.6798289078184523

## OpenCV – GMM/MOG

### MOG

In [53]:
# initializing subtractor 
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False, varThreshold=35)
  
for i in range(300, 1100):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)    
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)

  
    # applying on each frame
    fgmask = fgbg.apply(IG)  
  
    cv2.imshow('frame', fgmask)
    cv2.waitKey(10)
    
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((fgmask == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((fgmask == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((fgmask == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((fgmask == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()


In [ ]:
P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.831519174001968

### KNN

In [ ]:
# initializing subtractor 
fgbg = cv2.createBackgroundSubtractorKNN(detectShadows=False)
  
for i in range(300, 1100, 2):
    IG = I_gray('../common/pedestrian/input/in%06d.jpg' % i)    
    I_ref = I_gray('../common/pedestrian/groundtruth/gt%06d.png' % i)

  
    # applying on each frame
    fgmask = fgbg.apply(IG)  
  
    cv2.imshow('frame', fgmask)
    cv2.waitKey(10)
    
    f = open('../common/pedestrian/temporalROI.txt','r') # open file
    line = f.readline() # read line
    roi_start, roi_end = line.split() # split line
    roi_start = int(roi_start) # conversion to int
    roi_end = int(roi_end) # conversion to int

    if (i >= roi_start and i <= roi_end):
        TP_M = np.logical_and((fgmask == 255),(I_ref == 255)) # logical product of the matrix elements
        TP_S = np.sum(TP_M) # sum of the elements in the matrix
        TP = TP + TP_S # update of the global indicator

        TN_M = np.logical_and((fgmask == 0),(I_ref == 0)) # logical product of the matrix elements
        TN_S = np.sum(TN_M) # sum of the elements in the matrix
        TN = TN + TN_S # update of the global indicator

        FP_M = np.logical_and((fgmask == 255),(I_ref == 0)) # logical product of the matrix elements
        FP_S = np.sum(FP_M) # sum of the elements in the matrix
        FP = FP + FP_S # update of the global indicator

        FN_M = np.logical_and((fgmask == 0),(I_ref == 255)) # logical product of the matrix elements
        FN_S = np.sum(FN_M) # sum of the elements in the matrix
        FN = FN + FN_S # update of the global indicator
cv2.destroyAllWindows()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
F1

0.7415751268310554

## Wykorzystanie sieci neuronowej do segmentacji obiektów pierwszoplanowych

In [ ]:
import cv2
import os

image_folder = '../common/pedestrian/input/'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
print(height, width, layers)

video = cv2.VideoWriter(video_name, 0, 50, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

240 360 3
